In [1]:
from diffusers import (
    StableDiffusionControlNetPipeline,
    ControlNetModel,
    StableDiffusionImg2ImgPipeline
)
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
import imageio.v2 as imageio

In [2]:
# --- Setup device ---
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# --- Load ControlNet for scribble ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
# --- Prompt and image setup ---
sketch_path = "/content/boy_sketch.jpg"
sketch = Image.open(sketch_path).convert("RGB")

prompt_base = "anime style boy, vibrant colors, highly detailed"
env_start = "in a sunny forest"
env_end = "in a rainy forest"

In [5]:
# --- Output folder
os.makedirs("frames", exist_ok=True)

In [6]:
# --- Generate first frame ---
def generate_first_frame(sketch_img, prompt):
    print(f"Generating first frame with prompt: {prompt}")
    result = pipe(prompt=prompt, image=sketch_img, num_inference_steps=30)
    return result.images[0]

first_frame = generate_first_frame(sketch, f"{prompt_base}, {env_start}")
first_frame.save("frames/frame_000.png")

Generating first frame with prompt: anime style boy, vibrant colors, highly detailed, in a sunny forest


  0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
# Clear pipeline and free memory
del pipe, controlnet
torch.cuda.empty_cache()

In [8]:
# --- Load Img2Img pipeline with LoRA ---
img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
).to(device)

img2img_pipe.load_lora_weights("aionthegrind/anime-lora")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

anime-lora-model.safetensors:   0%|          | 0.00/89.7M [00:00<?, ?B/s]

No LoRA keys associated to UNet2DConditionModel found with the prefix='unet'. This is safe to ignore if LoRA state dict didn't originally have any UNet2DConditionModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [9]:
# --- Setup CLIP for similarity check ---
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def clip_similarity(img1, img2):
    inputs = clip_processor(images=[img1, img2], return_tensors="pt").to(device)
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        similarity = torch.cosine_similarity(features[0:1], features[1:2]).item()
    return similarity


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
# --- Smooth prompt blending ---
def blend_prompt(alpha):
    return (
        f"{prompt_base}, anime screencap, "
        f"{env_start}:{1 - alpha:.2f}, {env_end}:{alpha:.2f}"
    )

In [11]:
# --- Retry logic function ---
def generate_frame_with_retries(
    img2img_pipe, prev_frame, prompt, base_seed, frame_idx, max_retries, threshold
):
    retries = 0
    while retries <= max_retries:
        seed = base_seed + frame_idx + retries * 1000  # change seed on retry
        generator = torch.manual_seed(seed)

        new_frame = img2img_pipe(
            prompt=prompt,
            image=prev_frame,
            strength=0.65,
            guidance_scale=7.5,
            num_inference_steps=25,
            generator=generator,
        ).images[0]

        sim = clip_similarity(prev_frame, new_frame)
        if sim >= threshold:
            return new_frame, sim
        else:
            print(f"Retry {retries+1}/{max_retries}: similarity={sim:.3f} < {threshold} for frame {frame_idx}")
            retries += 1

    print(f"Warning: Low similarity persisted after {max_retries} retries at frame {frame_idx}")
    return new_frame, sim

In [12]:
# --- Simple temporal smoothing ---
def temporal_smoothing(img_prev, img_new, alpha=0.7):
    img_prev = img_prev.convert("RGBA")
    img_new = img_new.convert("RGBA")
    blended = Image.blend(img_new, img_prev, alpha=alpha)
    return blended.convert("RGB")

In [13]:
# --- Frame generation loop ---
frames = [first_frame]
num_frames = 40
base_seed = 42
similarity_threshold = 0.85
max_retries = 3

print("Generating frames with retry logic and temporal smoothing...")

for i in range(1, num_frames):
    alpha = i / num_frames
    blended_prompt = blend_prompt(alpha)

    new_frame, similarity = generate_frame_with_retries(
        img2img_pipe, frames[-1], blended_prompt, base_seed, i, max_retries, similarity_threshold
    )

    smoothed_frame = temporal_smoothing(frames[-1], new_frame, alpha=0.7)

    smoothed_frame.save(f"frames/frame_{i:03d}.png")
    frames.append(smoothed_frame)

    print(f"Frame {i} saved with similarity {similarity:.4f}")

Generating frames with retry logic and temporal smoothing...


  0%|          | 0/16 [00:00<?, ?it/s]

Frame 1 saved with similarity 0.9459


  0%|          | 0/16 [00:00<?, ?it/s]

Frame 2 saved with similarity 0.9719


...

Frame 37 saved with similarity 0.9327


  0%|          | 0/16 [00:00<?, ?it/s]

Frame 38 saved with similarity 0.9347


  0%|          | 0/16 [00:00<?, ?it/s]

Frame 39 saved with similarity 0.9348


In [14]:
# --- Final CLIP similarity between first and last frame ---
final_similarity = clip_similarity(frames[0], frames[-1])
print(f"CLIP similarity between first and last frame: {final_similarity:.4f}")

CLIP similarity between first and last frame: 0.8549


In [15]:
# --- Create video from frames ---
video_path = "output_animation.mp4"
fps = 8
print(f"Creating video at {fps} fps...")

with imageio.get_writer(video_path, fps=fps) as writer:
    for i in range(len(frames)):
        img = imageio.imread(f"frames/frame_{i:03d}.png")
        writer.append_data(img)

Creating video at 8 fps...


In [16]:
# --- Display video in notebook ---
from IPython.display import HTML
from base64 import b64encode
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")